# 1. Import

In [1]:
# Basic Library
import os
import pandas as pd
import numpy as np
import pickle
from itertools import permutations, combinations
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, QuantileTransformer

# HP Tuning
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from optuna.visualization import plot_contour, plot_optimization_history
from optuna.visualization import plot_parallel_coordinate, plot_slice, plot_param_importances

# Modeling
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor
import xgboost as xgb
from catboost import CatBoostRegressor, Pool

In [2]:
def create_dir(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
        print("Created Directory :", dir)
    else:
        print("Directory already existed :", dir)
create_dir("../pickle")
create_dir("../model")
create_dir("../submission")

Directory already existed : ../pickle
Directory already existed : ../model
Directory already existed : ../submission


In [3]:
train = pd.read_csv("../data/train_f2.csv")
test = pd.read_csv("../data/test_f2.csv")
submission = pd.read_csv("../data/sample_submission.csv")

In [4]:
rows_train = train.shape[0] # 주어진 train data의 row 수
rows_test = test.shape[0] # 주어진 test data의 row 수
num_trial = 100 # 파라미터 튜닝을 몇 번 진행하는지의 수
splits_hp = 5 # 파라미터 튜닝을 진행할 때의 kfold 수
splits_tr = 15 # 모델 트레이닝을 진행할 때의 kfold 수
basic_seed = 42 # default seed
num_seed_hp = 3 # 파라미터 튜닝 seed 개수
num_seed_tr = 10 # 트레이닝 seed 개수
sel_seed = 3 # 선택할 seed 개수

In [5]:
pred_dict = {}
pred_test_dict = {}

# 2. Catboost

In [6]:
del_cols = train.columns[train.columns.str.startswith("Gender#")]
train = train.drop(del_cols, axis=1)
test = test.drop(del_cols, axis=1)

In [7]:
train_x = train.drop(['Target'], axis=1) # 데이터 나누기
train_y = train['Target']
test_x = test.copy()

In [8]:
cat_features = ["Gender"]

In [9]:
def cat_objective(trial: Trial) -> float:
    score_hp = []
    for seed_hp in [0, 42]:
        params_cat = {
            "cat_features": cat_features,
            "random_state": seed_hp,
            "loss_function": "MAE",
            "eval_metric": "MAE",
            "iterations": 10000,
            "od_type": "iter",
            "od_wait": trial.suggest_int("od_wait", 50, 500),
            "learning_rate": trial.suggest_uniform("learning_rate", 1e-2, 1e-1), # default=0.03, range=[0,1]
            "colsample_bylevel":trial.suggest_float("colsample_bylevel", 0.3, 0.8),
            "bagging_temperature" :trial.suggest_loguniform('bagging_temperature', 1e-1, 1e+2),
            "random_strength" :trial.suggest_int('random_strength', 0, 30),
            "depth": trial.suggest_int("depth", 4, 7),
            "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-5, 1e+0), # default=3, range=[0,1]
            "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 30),
            "max_bin": trial.suggest_int("max_bin", 300, 500),
        }

        kfold = StratifiedKFold(n_splits=splits_hp, random_state=seed_hp, shuffle=True) # Cross-validation cv=5
        cv = np.zeros(rows_train)

        for n, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):

            x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
            y_train, y_val = train_y.iloc[train_idx].values, train_y.iloc[val_idx].values

            catmodel = CatBoostRegressor(**params_cat)
                                                                                            # 진행상황 보고싶을때 False를 100으로
            catmodel.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=50, verbose=False) 
            cv[val_idx] = catmodel.predict(x_val)
            
        score_hp.append(mean_absolute_error(train_y, cv))
    
    np.mean(score_hp)
    return np.mean(score_hp)

In [10]:
sampler = TPESampler(seed=basic_seed)
cat_study = optuna.create_study(study_name="cat_parameter_opt", direction="minimize", sampler=sampler)
cat_study.optimize(cat_objective, n_trials=num_trial)

cat_best_hyperparams = cat_study.best_trial.params
cat_base_hyperparams = {'loss_function': 'MAE', 'eval_metric': 'MAE', 'cat_features': cat_features, 'random_state': basic_seed,
                        'od_type': 'iter', 'iterations':10000}
cat_best_hyperparams.update(cat_base_hyperparams)

with open('../pickle/cat_best_hyperparams.pickle', 'wb') as fw:
    pickle.dump(cat_best_hyperparams, fw)
print("The best hyperparameters are:\n", cat_best_hyperparams)

[I 2022-03-31 02:00:56,379] A new study created in memory with name: cat_parameter_opt
[I 2022-03-31 02:15:13,400] Trial 0 finished with value: 1.47707885011161 and parameters: {'od_wait': 218, 'learning_rate': 0.09556428757689246, 'colsample_bylevel': 0.6659969709057025, 'bagging_temperature': 6.2513735745217485, 'random_strength': 4, 'depth': 4, 'l2_leaf_reg': 1.951722464144947e-05, 'min_data_in_leaf': 26, 'max_bin': 420}. Best is trial 0 with value: 1.47707885011161.
[I 2022-03-31 03:27:34,463] Trial 1 finished with value: 1.47614772529051 and parameters: {'od_wait': 369, 'learning_rate': 0.011852604486622221, 'colsample_bylevel': 0.7849549260809972, 'bagging_temperature': 31.428808908401084, 'random_strength': 6, 'depth': 4, 'l2_leaf_reg': 8.260808399079601e-05, 'min_data_in_leaf': 10, 'max_bin': 405}. Best is trial 1 with value: 1.47614772529051.
[I 2022-03-31 04:03:04,576] Trial 2 finished with value: 1.470790153326242 and parameters: {'od_wait': 244, 'learning_rate': 0.036210622

CatBoostError: bad allocation

In [ ]:
optuna.visualization.matplotlib.plot_param_importances(cat_study);

In [ ]:
optuna.visualization.matplotlib.plot_slice(cat_study);

In [ ]:
# with open('../pickle/cat_best_hyperparams.pickle', 'rb') as fw:
#     cat_best_hyperparams = pickle.load(fw)

In [ ]:
lucky_seeds = np.random.randint(0, 1000, num_seed_tr)

for i, seed in enumerate(lucky_seeds):

    kfold = StratifiedKFold(n_splits=splits_tr, random_state=seed, shuffle=True) # CV 늘려가면서 하기
    cv = np.zeros(rows_train)
    pred_test = np.zeros(rows_test)

    for n, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):
        
        x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
        y_train, y_val = train_y.iloc[train_idx].values.ravel(), train_y.iloc[val_idx].values.ravel()

        catmodel = CatBoostRegressor(**cat_best_hyperparams)
        catmodel.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=50, verbose=False)
        
        cv[val_idx] = catmodel.predict(x_val)
        pred_test += catmodel.predict(test_x) / splits_tr
        
    pred_dict['cat'+str(seed)] = cv
    pred_test_dict['cat'+str(seed)] = pred_test
    print(f'seed {seed}', 'mean_absolute_error :', mean_absolute_error(train_y, cv))

In [ ]:
pred_dict

In [ ]:
pred = (pred_dict['cat770'] + pred_dict['cat578'] + pred_dict['cat201']) / 3

In [ ]:
pred_test = (pred_test_dict['cat770'] + pred_test_dict['cat578'] + pred_test_dict['cat201']) / 3

In [ ]:
mean_absolute_error(train_y, np.round(pred))

# 3. Export

In [ ]:
def sort_dict(model, pred_dict, pred_test_dict):
    pred_dict_local = {}
    for key, value in pred_dict.items():
        if model in key:
            pred_dict_local[key]=value

    pred_test_dict_local = {}
    for key, value in pred_test_dict.items():
        if model in key:
            pred_test_dict_local[key]=value

    pred_dict_new_local = dict(sorted(
        pred_dict_local.items(), 
        key=lambda x:mean_absolute_error((train_y), list(x[1])), reverse=False)[:sel_seed])
    pred_test_dict_new_local = {}
    for key, value in pred_dict_new_local.items():
        pred_test_dict_new_local[key]=pred_test_dict_local[key]
        
    return pred_dict_new_local, pred_test_dict_new_local

In [ ]:
def save_dict(model, pred_dict, pred_test_dict):
    with open('../pickle/pred_dict_'+model+'.pickle', 'wb') as fw:
        pickle.dump(pred_dict, fw)
    with open('../pickle/pred_test_dict_'+model+'.pickle', 'wb') as fw:
        pickle.dump(pred_test_dict, fw)

In [ ]:
pred_dict_cat, pred_test_dict_cat = sort_dict('cat', pred_dict, pred_test_dict)
save_dict('cat', pred_dict_cat, pred_test_dict_cat)

In [ ]:
with open('../pickle/autosklearn_cv10_seed0.pickle', 'rb') as fw:
    pred_test_2 = pickle.load(fw)

In [ ]:
submission.Target = np.round(pred_test).astype(int)

In [ ]:
submission_name = '20220329'
submission_number = '3'
submission.to_csv(f'../submission/{submission_name}-{submission_number}.csv', index = False)